# Problems of Chapter 1

## 1.1

In [ ]:
from collections import Counter
import tabulate
import re
from string import ascii_lowercase


ciphertext = """lrvmnir bpr sumvbwvr jx bpr lmiwv yjeryrkbi jx qmbm wi
bpr xjvni mkd ymibrut jx irhx wi bpr riirkvr jx
ymbinlmtmipw utn qmumbr dj w ipmhh but bj rhnvwdmbr bpr
yjeryrkbi jx bpr qmbm mvvjudwko bj yt wkbrusurbmbwjk
lmird jk xjubt trmui jx ibndt
wb wi kjb mk rmit bmiq bj rashmwk rmvp yjeryrkb mkd wbi
iwokwxwvmkvr mkd ijyr ynib urymwk nkrashmwkrd bj ower m
vjyshrbr rashmkmbwjk jkr cjnhd pmer bj lr fnmhwxwrd mkd
wkiswurd bj invp mk rabrkb bpmb pr vjnhd urmvp bpr ibmbr
jx rkhwopbrkrd ywkd vmsmlhr jx urvjokwgwko ijnkdhrii
ijnkd mkd ipmsrhrii ipmsr w dj kjb drry ytirhx bpr xwkmh
mnbpjuwbt lnb yt rasruwrkvr cwbp qmbm pmi hrxb kj djnlb
bpmb bpr xjhhjcwko wi bpr sujsru msshwvmbwjk mkd
wkbrusurbmbwjk w jxxru yt bprjuwri wk bpr pjsr bpmb bpr
riirkvr jx jqwkmcmk qmumbr cwhh urymwk wkbmvb"""

counter = Counter(dict.fromkeys(ascii_lowercase, 0))
counter.update(re.sub(r"\W+", "", ciphertext))
total = sum(counter.values())

# frequency distribution of ciphertext
tabulate.tabulate(
    [[letter, counter[letter] / total] for letter in sorted(counter)],
    headers=["letter", "frequency"],
    floatfmt=".4f",
    tablefmt="html",
)

In [ ]:
from operator import itemgetter

from ancient import SubstitutionCipher

# letter frequencies of the English language
letter_frequencies = [
    ("a", 0.0817),
    ("b", 0.0150),
    ("c", 0.0278),
    ("d", 0.0425),
    ("e", 0.1270),
    ("f", 0.0223),
    ("g", 0.0202),
    ("h", 0.0609),
    ("i", 0.0697),
    ("j", 0.0015),
    ("k", 0.0403),
    ("l", 0.0077),
    ("m", 0.0675),
    ("n", 0.0241),
    ("o", 0.0751),
    ("p", 0.0193),
    ("q", 0.0010),
    ("r", 0.0599),
    ("s", 0.0633),
    ("t", 0.0906),
    ("u", 0.0276),
    ("v", 0.0098),
    ("w", 0.0236),
    ("x", 0.0015),
    ("y", 0.0197),
    ("z", 0.0007),
]

letter_frequencies.sort(key=itemgetter(1), reverse=True)
table = {x: y for (x, _), (y, _) in zip(letter_frequencies, counter.most_common())}

# unfortunately more than half of the letters have to be tweaked manually
tweaked_letters = {
    "z": "g",
    "q": "f",
    "x": "a",
    "l": "h",
    "k": "q",
    "p": "s",
    "f": "x",
    "u": "n",
    "w": "c",
    "n": "k",
    "o": "j",
    "i": "w",
    "m": "y",
    "g": "o",
    "b": "l",
    "y": "t",
    "v": "e",
    "j": "z",
}

table.update(tweaked_letters)
print(SubstitutionCipher(table).decode(ciphertext))

## 1.2

In [ ]:
from ancient import CaesarCipher

# to break the Caesar Cipher we need to correctly identify only one letter

ciphertext = "xultpaajcxitltlxaarpjhtiwtgxktghidhipxciwtvgtpilpitghlxiwiwtxgqadds"
counter = Counter(ciphertext)

x = max(letter_frequencies, key=itemgetter(1))[0]  # most common English letter
y = counter.most_common(1)[0][0]  # most common letter in ciphertext
k = (ord(y) - ord(x)) % 26

print(f"{k = }")
print(CaesarCipher(k).decode(ciphertext))

## 1.4


1. key space: $128^8 = 2^{56}$

2. key length: $7 \cdot 8 = 56$ bits

3. Actual key length is $56$ bits, just like before. The *effective* key length is $\left\lceil \log_2 {26^8} \right\rceil = {38}$ bits.

4. Again, we are interested in achieving the effective key length:

   - $\left\lceil 128 / 7 \right\rceil = 19$ characters

   - $\left\lceil \log_{26} 2^{128} \right\rceil = 28$ letters

## 1.6

In [ ]:
def inv(a: int, m: int) -> int:
    for x in range(1, m):
        if a * x % m == 1:
            return x
    raise ArithmeticError(f"{a=} does not have inverse module {m=}")


print(f"1/5 = {inv(5, 13)} mod 13")
print(f"1/5 = {inv(5, 7)} mod 7")
print(f"3 * 2/5 = {3 * 2 * inv(5, 7) % 7 } mod 7")


## 1.8

In [ ]:
for m in (11, 12, 13):
    print(f"1/5 = {inv(5, m)} mod {m}")

## 1.10

In [ ]:
import math

# https://en.wikipedia.org/wiki/Totative

tabulate.tabulate(
    [
        [m, totatives := {n for n in range(m) if math.gcd(n, m) == 1}, len(totatives)]
        for m in (4, 5, 9, 26)
    ],
    headers=["m", "totatives of m", "phi(m)"],
    tablefmt="html",
)

## 1.12

Let $x, y, a, b \in \mathbb{Z}_{30}$<br>
Encryption: $e_k(x) = y \equiv a \cdot x + b \mod 30$.<br>
Decryption: $d_k(y) = x \equiv a^{−1}  \cdot (y − b) \mod 30$.<br>
with the key: $k = (a, b)$, which has the restriction: $\gcd(a, 30) = 1$.

In [ ]:
from ancient import AffineCipher


def phi(m: int) -> int:
    return sum(1 for n in range(m) if math.gcd(n, m) == 1)


german_alphabet = ascii_lowercase + "äöüß"
m = len(german_alphabet)
print(f"key space = {phi(m) * m}")

key = (17, 1)
ciphertext = "äußwß"
print(AffineCipher(key, german_alphabet).decode(ciphertext))

## 1.14

In [ ]:
a1, b1 = 3, 5
a2, b2 = 11, 7
a3, b3 = (a1 * a2) % 26, (a2 * b1 + b2) % 26

assert a3 in {1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25}

print(f"{(a3, b3) = }")

In [ ]:
e1 = AffineCipher((a1, b1)).encode
e2 = AffineCipher((a2, b2)).encode
e3 = AffineCipher((a3, b3)).encode

assert e2(e1("K")) == e3("K")

In general, for $i$ number of applications of the affine cipher, we can express $a,b$ as:
$$
\begin{align*}
a &= \prod_{i = 1}^n a_i \mod 26\\
b &= \sum_{i = 1}^n \bigl( b_i \prod_{j = i + 1}^n a_j \bigr) \mod 26
\end{align*}
$$

In [ ]:
import random
from typing import NamedTuple


class AffineKey(NamedTuple):
    a: int
    b: int

    @classmethod
    def generate_random_key(cls) -> "AffineKey":
        a = random.choice([1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25])
        b = random.randint(1, 25)
        return cls(a, b)


k1 = AffineKey.generate_random_key()
k2 = AffineKey.generate_random_key()
k3 = AffineKey.generate_random_key()

e1 = AffineCipher(k1).encode
e2 = AffineCipher(k2).encode
e3 = AffineCipher(k3).encode


def e4(plaintext: str, *keys: AffineKey) -> str:
    a, b = 1, 0
    for ai, bi in reversed(keys):
        b += a * bi
        a *= ai
    return AffineCipher((a, b)).encode(plaintext)


assert e3(e2(e1("K"))) == e4("K", k1, k2, k3)